<a href="https://colab.research.google.com/github/vieiramesquita/GEARS/blob/master/TeroPoDa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Name
TeroPoDa - Time Series Extraction for Polygonal Data

### Description
Toolkit created to extract time series information from Sentinel 2 data stored in Earth Engine

### Author
  Vinícius Vieira Mesquita - vieiramesquita@gmail.com

### Version
  1.0.5

### Import main libraries

Run the following cell to import the main API's into your session.

In [30]:
import sys
import ee
import pandas as pd
import os
import time
from joblib import Parallel, delayed

### Authenticate and initialize

Run the `ee.Authenticate` function to authenticate your access to Earth Engine servers and `ee.Initialize` to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. Follow the instructions printed to the cell.

In [31]:
# Trigger the authentication flow.
#ee.Authenticate()

# Initialize the library.
ee.Initialize()


### Get the NDVI Time Series from Earth Engine 

Function responsible to get the time series of Sentinel 2 data throught Earth Engine.

This function needs a `geometry` object in the `ee.Feature()` formart and the choosed vetor propertie ID as the `id_field`.



In [ ]:
def getTimeSeries(geometry,id_field,bestEffort=False):
  
  def mask_img(img):

    satName = ee.String(img.get('SPACECRAFT_NAME'))
  
    maskExp = "(b('MSK_CLDPRB') < 5 && b('SCL') != 3 && b('SCL') != 10)"

    #maskExp = "b('Q60') == 0"

    mask = img.expression(maskExp).add(img.lte(0));

    ndvi = img.updateMask(mask).normalizedDifference(['B8','B4']).select([0],['NDVI'])
  
    return (img.addBands(ndvi, None, True)
      .set({'system:time_start':img.get('system:time_start'),'satelite':satName}))

  def reduceData(img):

    img = ee.Image(img)

    imgDate = ee.Date(ee.Number(img.get('system:time_start')))
    orgDate = (ee.String(ee.Number(imgDate.get('year')).toInt().format())
      .cat('-')
      .cat(ee.String(ee.Number(imgDate.get('month')).toInt().format()))
      .cat('-')
      .cat(ee.String(ee.Number(imgDate.get('day')).toInt().format()))
      )
    
    reducers = (ee.Reducer.mean()
        .combine(**{'reducer2': ee.Reducer.stdDev(),'sharedInputs':True,})
        .combine(**{'reducer2': ee.Reducer.count(),'sharedInputs':True}))

    if bestEffort == False: 
      series = img.reduceRegion(reducers,ee.Feature(geometry).geometry(), 10,None,None,False,1e13,16)
    else:
      series = img.reduceRegion(reducers,ee.Feature(geometry).geometry(), None,None,None,True,1e13,16)
    
    return (ee.Feature(geometry)
      .set('id',ee.String(img.id()))
      .set('date',orgDate)
      .set('satelite',img.get('satelite'))
      .set('MGRS_TILE',img.get('MGRS_TILE'))      
      .set('AREA_HA',ee.Feature(geometry).get('AREA_HA'))
      .set(id_field,ee.Feature(geometry).get(id_field))
      .set('NDVI_mean',ee.Number(ee.Dictionary(series).get('NDVI_mean')))
      .set('NDVI_stdDev',ee.Number(ee.Dictionary(series).get('NDVI_stdDev')))
      .set('Pixel_Count',ee.Number(ee.Dictionary(series).get('NDVI_count')))
      #.toDictionary()
    )

  def toDict(feat):
    return ee.Feature(feat).toDictionary()

  #imgCol =  (ee.ImageCollection("COPERNICUS/S2_HARMONIZED").filterBounds(geometry.geometry())
  #  .map(mask_img))

  imgCol = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(geometry.geometry())
    .map(mask_img))

  Coll_fill = (imgCol.toList(imgCol.size()).map(reduceData)
    .filter(ee.Filter.notNull(['NDVI_mean']))
    .map(toDict)
    )
    
  return Coll_fill

### Build and Structure the Time Series library

Function responsible to build and structure the time series library.

In [33]:
def build_time_series(index,obj,id_field,outfile,asset,bestEffort=False):

  samples = ee.FeatureCollection(asset).select(id_field)
  #samples = ee.FeatureCollection('users/vieiramesquita/TNC_Vectors/monitoring_polygons_TNC').select(id_field)

  df = pd.DataFrame()

  start_time_obj = time.time()

  samples_split = samples.filter(ee.Filter.eq(id_field,obj)).first()

  point_series = getTimeSeries(ee.Feature(samples_split),id_field,bestEffort).getInfo()
  
  for item in point_series:
    df  = pd.concat([df,pd.DataFrame(item,index=[0])])
  
  hdr = False if os.path.isfile(outfile) else True
  
  df['NDVI_mean'] = df['NDVI_mean'].round(decimals=4) 
  df['NDVI_stdDev'] = df['NDVI_stdDev'].round(decimals=4) 

  df.to_csv(outfile,mode='a',header = hdr,index = False,sep =';')

  time_spended = round(time.time() - start_time_obj, 3)

  print(f'Index {index} - Object [{obj}] procesed in {round(time.time() - start_time_obj, 3)} seconds')

  if df.shape[0] > 0: 
    return True,time_spended
  elif float(df['AREA_HA']) < 0.01:
    return None,None
  else:
    return False,None

### Check the Time Series library

Function responsible to check the consistency of the time series library.

In [34]:
def build_time_series_reroll(index,obj,id_field,outfile,asset,checker=False):

  obj = int(obj)

  if checker is True:

    df_check = pd.read_csv(outfile,sep =';')

    if obj in list(df_check[id_field].values):

      print(f' Object [{obj}] was found in the file. Skipping..')
      return {'errors':None, 'time': 0}

  errors = None
  time = None

  try:
    check = build_time_series(index,obj,id_field,outfile,asset)
    time = check[1]

    if check[0] == False:
      print('raised')
      raise
    if check[0] == None:
      return {'errors':'ignore' ,'time': 'ignore'}
      
  except:

    try:

      print(f'Index {index} - Request [{obj}] fails. Trying the best effort!')

      check = build_time_series(index,obj,id_field,outfile,asset,True)

      if check[0] == False:
        print('raised')
        raise
      
      if check[0] == None:
        return {'errors':'ignore' ,'time': 'ignore'}
      
    except:

      print(f'Index {index} - Request [{obj}] expired. Sending it to the error list!')
        
      errors = obj

  return {'errors':errors ,'time': time}

### Build the Polygon List file

Function responsible to write a text file contaning each Polygon ID used to extract the time series.

In [35]:
def build_id_list(asset,id_field):
  
  samples = ee.FeatureCollection(asset).select(id_field)
  
  sample_size = int(samples.size().getInfo())

  if sample_size > 50000:
    samples_list = samples.toList(50000)
  else:
    samples_list = samples.toList(samples.size())

  with open('/content/polygonList.txt', "w") as polygon_file:

    def get_ids(feat):
      return ee.Feature(feat).get(id_field)

    samples_list_slice = samples_list.map(get_ids).sort().getInfo()

    for polygon in samples_list_slice:
      polygon_file.write(str(polygon)+ '\n')
  

### Run

Function responsible to catch argument information and start run the process.

In [36]:
def run(asset,id_field,output_name):
  start_time = time.time()

  mon_Polygons_text = open("/content/polygonList.txt","r")
  mon_Polygons = mon_Polygons_text.readlines()

  mon_Polygons = [int(name) for name in mon_Polygons]

  start_obj = 1

  total = len(mon_Polygons)

  print(f'Number of objects to process: {total}')

  if total > 1000:
    print('Go take a coffee and watch a series... it will take a while!')
  
  #out_dir = r'E:\TrabalhosLAPIG\ToolkitNDVI_S2_TNC'
  #output_name = os.path.join(out_dir,'TNC_S2_NDVI_Monitoring.csv')
  #id_field = 'ID_POL'

  list_num = mon_Polygons[start_obj:total]

  #list_num = mon_Polygons[1:50]

  first_dict = [{'errors':'ignore' ,'time': 'ignore'}]
  check_file = True

  if os.path.exists(output_name) is False:

    check_file = False
    print('Creating the main file..')
    first_dict = build_time_series_reroll(0,int(mon_Polygons[0]),id_field,output_name,asset)

  worker_args = [
    (mon_Polygons.index(obj),obj,id_field,output_name,asset,check_file) \
    for obj in list_num
  ]
  
  infos = Parallel(n_jobs=16, backend='multiprocessing')(delayed(build_time_series_reroll)(*args) for args in worker_args)

  if check_file is True:
    first_dict = {'time': 0}

  time_list = [first_dict['time']] + [item['time'] for item in infos if item['time'] != None]

  errors_list = [item['errors'] for item in infos if item['errors'] != None]

  with open('/content/errors_polygon.txt', "w") as errors_file:  
    for polygon in errors_list:
      errors_file.write(str(polygon)+ '\n')

  print(f'The average processing time was {round(pd.DataFrame(time_list).mean()[0],2)} seconds')
  print(f'Processing finished. All the work took {round(time.time() - start_time,3)} seconds to complete')

In [ ]:
if __name__ == '__main__':

  asset = 'users/vieiramesquita/TNC_Vectors/monitoring_polygons_TNC'
  id_field = 'ID_POL'
  output_name = '/content/TNC_S2_NDVI_Monitoring.csv'

  if os.path.exists('/content/polygonList.txt') is False:
    build_id_list(asset,id_field)

  run(asset,id_field,output_name)




Index 5107 - Object [10217] procesed in 16.48 seconds
Index 5113 - Object [10223] procesed in 6.989 seconds
Index 5112 - Object [10222] procesed in 9.957 seconds
